In [9]:
import pickle, os, sys, glob
import pandas as pd
import numpy as np

In [10]:
def wake_events(data, time_point):
    """
    Counts the transitions from False to True in a given list of booleans.

    :param data: list of booleans
    :return: count of False to True transitions
    """
    data = data['Hours'][time_point]['sls all'] == 'W'
    count = 0
    # Iterate over the list from the second element to the end
    for i in range(1, len(data)):
        # Check if the current value is True and the previous value is False
        if data[i] and not data[i - 1]:
            count += 1
    return count

def extract_pt(file_path):
    # Split the path by '/' and get the last part, which is the filename with extension
    filename_with_ext = file_path.split('/')[-1]
    
    # Now split the filename by '.' to separate the extension and select the part before the extension
    filename = filename_with_ext.split('.')[0]
    
    return filename

['sleep_onset',
 'Hour 0 W',
 'Hour 0 R',
 'Hour 0 N1',
 'Hour 0 N2',
 'Hour 0 N3',
 'Hour 1 W',
 'Hour 1 R',
 'Hour 1 N1',
 'Hour 1 N2',
 'Hour 1 N3',
 'Hour 2 W',
 'Hour 2 R',
 'Hour 2 N1',
 'Hour 2 N2',
 'Hour 2 N3',
 'Hour 3 W',
 'Hour 3 R',
 'Hour 3 N1',
 'Hour 3 N2',
 'Hour 3 N3',
 'Wake Events Hour 0',
 'Wake Events Hour 1',
 'Wake Events Hour 2',
 'Wake Events Hour 3',
 'Hour 0 Delta EEG C3-A2',
 'Hour 0 Theta EEG C3-A2',
 'Hour 0 Alpha EEG C3-A2',
 'Hour 0 Sigma EEG C3-A2',
 'Hour 0 Beta EEG C3-A2',
 'Hour 0 Gamma EEG C3-A2',
 'Hour 0 Delta EEG C4-A1',
 'Hour 0 Theta EEG C4-A1',
 'Hour 0 Alpha EEG C4-A1',
 'Hour 0 Sigma EEG C4-A1',
 'Hour 0 Beta EEG C4-A1',
 'Hour 0 Gamma EEG C4-A1',
 'Hour 0 Delta EEG O1-A2',
 'Hour 0 Theta EEG O1-A2',
 'Hour 0 Alpha EEG O1-A2',
 'Hour 0 Sigma EEG O1-A2',
 'Hour 0 Beta EEG O1-A2',
 'Hour 0 Gamma EEG O1-A2',
 'Hour 0 Delta EEG O2-A1',
 'Hour 0 Theta EEG O2-A1',
 'Hour 0 Alpha EEG O2-A1',
 'Hour 0 Sigma EEG O2-A1',
 'Hour 0 Beta EEG O2-A1',
 'H

In [52]:
demo_path = '/home/sam/Downloads/anonymized data - Sheet1.csv'
pickle_shop = ['/home/sam/Classes/Stats/Consulting/EEG_Consulting/MJCohortG3/NJE0311.pkl']

df = pd.read_csv(demo_path)


# Create statistic storage columns
sleep_stages = ["W", "R", "N1", "N2", "N3"]
channels = ['EEG C3-A2', 'EEG C4-A1', 'EEG O1-A2', 'EEG O2-A1']
bands = ['Delta', 'Theta', 'Alpha', 'Sigma', 'Beta', 'Gamma']

new_cols = [[[['sleep_onset']]],
           [[[f'Hour {i} {s}' for s in sleep_stages] for i in range(4)]],
            [[[f'Wake Events Hour {i}' for i in range(4)]]],
           [[[f'Hour {i} {bp} {eeg}' for bp in bands] for eeg in channels] for i in range(4)]
           ]
# unnest
for i in range(3):
    new_cols = [item for sublist in new_cols for item in sublist]
new_cols

# Initialize new columns with None
new_cols_df = pd.DataFrame({col: np.nan for col in new_cols}, index=df.index)

# Step 3: Concatenate the new columns DataFrame with the original DataFrame
df = pd.concat([df, new_cols_df], axis=1)


# Initialize a dictionary to store the loaded data
loaded_data = {}

# Loop over each pickle file and load its contents
for pickle_file in pickle_shop:
    with open(pickle_file, 'rb') as file:
        # Load the pickle file content and store it in the dictionary
        key = extract_pt(pickle_file)
        loaded_data[key] = pickle.load(file)

# Now you can access the loaded data from the dictionary
print("Loaded data from pickle files:")



for key in loaded_data.keys():
    print(f"Patient {key} initiated sleep {loaded_data[key]['Time2Sleep']} seconds after lights off")
    row_index = df[df['ba_rid'] == key].index # find the correct row
    df.loc[row_index, 'sleep_onset'] = loaded_data[key]['Time2Sleep']
    
    for i in range(4):
        for stage in sleep_stages:
            print(f"In hour {i} they were in sleep stage {stage} for {loaded_data[key]['Hours'][i][stage]} seconds")
            df.loc[row_index, f'Hour {i} {stage}'] = loaded_data[key]['Hours'][i][stage]
            if stage == 'W':
                we = wake_events(loaded_data[key], i)
                print(f'Of this time, there were {we} unique wake events predicted')
                df.loc[row_index, f'Wake Events Hour {i}'] = we
            for band in bands:
                for j, eeg in enumerate(channels):
                    df.loc[row_index, f'Hour {i} {band} {eeg}'] = loaded_data[key]['Hours'][i]['BandPower'].loc[j,band]
    print('\n'*3)

display(df)

df.to_csv('/home/sam/Classes/Stats/Consulting/EEG_Consulting/MJCohortG3/AllData_MJCohortG3.csv', index=False)  

Loaded data from pickle files:
Patient NJE0311 initiated sleep 1680 seconds after lights off
In hour 0 they were in sleep stage W for 720 seconds
Of this time, there were 6 unique wake events predicted
In hour 0 they were in sleep stage R for 0 seconds
In hour 0 they were in sleep stage N1 for 540 seconds
In hour 0 they were in sleep stage N2 for 1770 seconds
In hour 0 they were in sleep stage N3 for 570 seconds
In hour 1 they were in sleep stage W for 330 seconds
Of this time, there were 7 unique wake events predicted
In hour 1 they were in sleep stage R for 0 seconds
In hour 1 they were in sleep stage N1 for 450 seconds
In hour 1 they were in sleep stage N2 for 1860 seconds
In hour 1 they were in sleep stage N3 for 960 seconds
In hour 2 they were in sleep stage W for 930 seconds
Of this time, there were 10 unique wake events predicted
In hour 2 they were in sleep stage R for 150 seconds
In hour 2 they were in sleep stage N1 for 660 seconds
In hour 2 they were in sleep stage N2 for 16

,Obs,ba_rid,T,V,P,Lightsoffclocktime,J,X,THCMinutesSleep,perTHCSleepMODG,...,Hour 3 Alpha EEG O1-A2,Hour 3 Sigma EEG O1-A2,Hour 3 Beta EEG O1-A2,Hour 3 Gamma EEG O1-A2,Hour 3 Delta EEG O2-A1,Hour 3 Theta EEG O2-A1,Hour 3 Alpha EEG O2-A1,Hour 3 Sigma EEG O2-A1,Hour 3 Beta EEG O2-A1,Hour 3 Gamma EEG O2-A1
0,3,NJA0281,0,0,0,22:13:18,0,0,.,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,NJA1172,0,0,0,21:32:06,0,0,.,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,NJA2367,0,0,0,22:14:49,0,0,.,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12,NJA4888,0,0,0,22:44:42,0,0,.,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14,NJB0110,0,0,0,.,0,0,.,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,219,NJS0766,.,1,0,22:30:18,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,221,NJT0398,.,1,1,22:18:57,1,0,117,0.75241,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
232,225,NJT0875,.,1,0,.,1,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,229,NJU0885,.,1,0,22:30:05,1,1,120,0.27491,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)  # None to display all rows
pd.set_option('display.max_columns', None)  # None to display all columns


df.iloc[row_index,:]

,Obs,ba_rid,T,V,P,Lightsoffclocktime,J,X,THCMinutesSleep,perTHCSleepMODG,Q,sleep_onset,Hour 0 W,Hour 0 R,Hour 0 N1,Hour 0 N2,Hour 0 N3,Hour 1 W,Hour 1 R,Hour 1 N1,Hour 1 N2,Hour 1 N3,Hour 2 W,Hour 2 R,Hour 2 N1,Hour 2 N2,Hour 2 N3,Hour 3 W,Hour 3 R,Hour 3 N1,Hour 3 N2,Hour 3 N3,Wake Events Hour 0,Wake Events Hour 1,Wake Events Hour 2,Wake Events Hour 3,Hour 0 Delta EEG C3-A2,Hour 0 Theta EEG C3-A2,Hour 0 Alpha EEG C3-A2,Hour 0 Sigma EEG C3-A2,Hour 0 Beta EEG C3-A2,Hour 0 Gamma EEG C3-A2,Hour 0 Delta EEG C4-A1,Hour 0 Theta EEG C4-A1,Hour 0 Alpha EEG C4-A1,Hour 0 Sigma EEG C4-A1,Hour 0 Beta EEG C4-A1,Hour 0 Gamma EEG C4-A1,Hour 0 Delta EEG O1-A2,Hour 0 Theta EEG O1-A2,Hour 0 Alpha EEG O1-A2,Hour 0 Sigma EEG O1-A2,Hour 0 Beta EEG O1-A2,Hour 0 Gamma EEG O1-A2,Hour 0 Delta EEG O2-A1,Hour 0 Theta EEG O2-A1,Hour 0 Alpha EEG O2-A1,Hour 0 Sigma EEG O2-A1,Hour 0 Beta EEG O2-A1,Hour 0 Gamma EEG O2-A1,Hour 1 Delta EEG C3-A2,Hour 1 Theta EEG C3-A2,Hour 1 Alpha EEG C3-A2,Hour 1 Sigma EEG C3-A2,Hour 1 Beta EEG C3-A2,Hour 1 Gamma EEG C3-A2,Hour 1 Delta EEG C4-A1,Hour 1 Theta EEG C4-A1,Hour 1 Alpha EEG C4-A1,Hour 1 Sigma EEG C4-A1,Hour 1 Beta EEG C4-A1,Hour 1 Gamma EEG C4-A1,Hour 1 Delta EEG O1-A2,Hour 1 Theta EEG O1-A2,Hour 1 Alpha EEG O1-A2,Hour 1 Sigma EEG O1-A2,Hour 1 Beta EEG O1-A2,Hour 1 Gamma EEG O1-A2,Hour 1 Delta EEG O2-A1,Hour 1 Theta EEG O2-A1,Hour 1 Alpha EEG O2-A1,Hour 1 Sigma EEG O2-A1,Hour 1 Beta EEG O2-A1,Hour 1 Gamma EEG O2-A1,Hour 2 Delta EEG C3-A2,Hour 2 Theta EEG C3-A2,Hour 2 Alpha EEG C3-A2,Hour 2 Sigma EEG C3-A2,Hour 2 Beta EEG C3-A2,Hour 2 Gamma EEG C3-A2,Hour 2 Delta EEG C4-A1,Hour 2 Theta EEG C4-A1,Hour 2 Alpha EEG C4-A1,Hour 2 Sigma EEG C4-A1,Hour 2 Beta EEG C4-A1,Hour 2 Gamma EEG C4-A1,Hour 2 Delta EEG O1-A2,Hour 2 Theta EEG O1-A2,Hour 2 Alpha EEG O1-A2,Hour 2 Sigma EEG O1-A2,Hour 2 Beta EEG O1-A2,Hour 2 Gamma EEG O1-A2,Hour 2 Delta EEG O2-A1,Hour 2 Theta EEG O2-A1,Hour 2 Alpha EEG O2-A1,Hour 2 Sigma EEG O2-A1,Hour 2 Beta EEG O2-A1,Hour 2 Gamma EEG O2-A1,Hour 3 Delta EEG C3-A2,Hour 3 Theta EEG C3-A2,Hour 3 Alpha EEG C3-A2,Hour 3 Sigma EEG C3-A2,Hour 3 Beta EEG C3-A2,Hour 3 Gamma EEG C3-A2,Hour 3 Delta EEG C4-A1,Hour 3 Theta EEG C4-A1,Hour 3 Alpha EEG C4-A1,Hour 3 Sigma EEG C4-A1,Hour 3 Beta EEG C4-A1,Hour 3 Gamma EEG C4-A1,Hour 3 Delta EEG O1-A2,Hour 3 Theta EEG O1-A2,Hour 3 Alpha EEG O1-A2,Hour 3 Sigma EEG O1-A2,Hour 3 Beta EEG O1-A2,Hour 3 Gamma EEG O1-A2,Hour 3 Delta EEG O2-A1,Hour 3 Theta EEG O2-A1,Hour 3 Alpha EEG O2-A1,Hour 3 Sigma EEG O2-A1,Hour 3 Beta EEG O2-A1,Hour 3 Gamma EEG O2-A1
65,60,NJE0311,1,1,0,20:21:43,2,2,.,0,3,1680.0,720.0,0.0,540.0,1770.0,570.0,330.0,0.0,450.0,1860.0,960.0,930.0,150.0,660.0,1650.0,210.0,270.0,630.0,330.0,2130.0,240.0,6.0,7.0,10.0,8.0,0.736402,0.111643,0.085017,0.034441,0.024497,0.008001,0.723213,0.120017,0.089904,0.036743,0.022697,0.007427,0.60505,0.217346,0.086321,0.041531,0.040096,0.009655,0.634974,0.199785,0.094101,0.037113,0.028224,0.005802,0.751508,0.109677,0.075887,0.030823,0.023148,0.008916,0.747786,0.114358,0.078002,0.030608,0.021719,0.007395,0.699472,0.178318,0.059924,0.030623,0.024805,0.00679,0.66456,0.193044,0.077836,0.033486,0.025579,0.005399,0.71144,0.104537,0.080208,0.046384,0.043812,0.013604,0.663397,0.124455,0.084646,0.050821,0.058492,0.018097,0.673163,0.154613,0.071363,0.044738,0.044824,0.011134,0.661062,0.149258,0.075361,0.042156,0.054302,0.017931,0.702947,0.118181,0.083192,0.04724,0.037826,0.010772,0.757168,0.094634,0.067293,0.040504,0.031387,0.009079,0.682655,0.175496,0.067178,0.034893,0.033074,0.006782,0.656931,0.166664,0.083079,0.044428,0.040134,0.008884
